In [1]:
import nltk
import sys
import time
import numpy as np
import pandas as pd
import sklearn.feature_extraction.text as text
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from IPython.display import display

In [48]:
data = pd.read_csv('../data/train.csv')
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [47]:
data = pd.read_csv('../data/train.csv')
sample = data.sample(frac=0.5)
sample.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
147703,4073aa637f9d3972,"Those were the official figures, which were al...",0,0,0,0,0,0
122660,90218c80b58ab9aa,I want to meet you one day and stab you to dea...,1,0,0,1,0,0
88128,ebbf3013d8581352,"Decide, that GW was the first, the eight, or t...",0,0,0,0,0,0
138124,e3260fb68d4c4019,"""\n\n Mixins - yes or no \n\nThe """"mixins"""" se...",0,0,0,0,0,0
131578,bff6526c7173ca01,"Ran, Taiwanese dog and his female dog.",0,0,0,0,0,0


In [3]:
TARGET_LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat','insult',	'identity_hate']

In [4]:
sample[TARGET_LABELS].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [5]:
sample.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,79786.000000,79786.000000,79786.000000,79786.000000,79786.000000,79786.000000
mean,0.096458,0.010478,0.053882,0.002983,0.050535,0.008598
std,0.295220,0.101825,0.225785,0.054535,0.219048,0.092326
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [7]:
tf = text.TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
X = tf.fit_transform(sample['comment_text'])

In [8]:
y = sample[TARGET_LABELS].values

In [9]:
type(sample['comment_text'])

pandas.core.series.Series

In [10]:
(X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size=.2)

# Naive Bayes

In [20]:
%%time
bnb = GridSearchCV(
    BernoulliNB(),
    param_grid={'alpha': np.logspace(-2., 2., 50)})
bnb.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: bad input shape (42552, 6)

In [13]:
confusion_matrix(bnb.predict(X_test), y_test)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [19]:
bnb.score(X_test)

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

# Multinomial Naive Bayes

In [21]:
%%time
mnb = GridSearchCV(
    MultinomialNB(),
    param_grid={'alpha': np.logspace(-2., 2., 50)})
mnb.fit(X_train, y_train)

ValueError: bad input shape (42552, 6)

In [ ]:
confusion_matrix(mnb.predict(X_test), y_test)

In [ ]:
f1_score(mnb.predict(X_test), y_test)

# SVM Linear

In [24]:
%%time
import random
random.seed(6)
# svm_svc = svm.LinearSVC()
svm_svc = OneVsRestClassifier(svm.LinearSVC())
svm_svc.fit(X_train, y_train)

CPU times: user 4.07 s, sys: 252 ms, total: 4.33 s
Wall time: 3.3 s


In [ ]:
confusion_matrix(svm_svc.predict(X_test), y_test)

In [26]:
svm_svc.score(X_test, y_test)

0.9185361574132097

# Logistic Regression

In [ ]:
%%time
log_reg = GridSearchCV(
    LogisticRegression(),
    param_grid={'C': [5, 10, 15, 20, 30]})
log_reg.fit(X_train, y_train)

In [ ]:
log_reg.best_params_

In [ ]:
confusion_matrix(log_reg.predict(X_test), y_test)

In [ ]:
f1_score(log_reg.predict(X_test), y_test)

# Naive Bayes SVM

In [52]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from scipy import sparse
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ['_r', '_clf'])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y==y_i].sum(0)
            return (p+1) / ((y==y_i).sum()+1)

        self._r = sparse.csr_matrix(np.log(pr(x,1,y) / pr(x,0,y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(C=self.C, dual=self.dual, n_jobs=self.n_jobs).fit(x_nb, y)
        return self

In [ ]:
y

In [43]:
%%time
nbsvm = OneVsRestClassifier(NbSvmClassifier(C=4, dual=True, n_jobs=-1))
nbsvm.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


CPU times: user 31.4 s, sys: 495 ms, total: 31.9 s
Wall time: 22.6 s


In [ ]:
confusion_matrix(nbsvm.predict(X_test), y_test)

In [53]:
nbsvm.score(X_test, y_test)

0.9214187241508961

In [56]:
X_test[0]

<1x245186 sparse matrix of type '<class 'numpy.float64'>'
	with 109 stored elements in Compressed Sparse Row format>

In [54]:
nbsvm.predict('Fuck you')

AttributeError: 'str' object has no attribute 'multiply'

In [44]:
f1_score(nbsvm.predict(X_test), y_test)

ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting.

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200, n_jobs=-1)
rf.fit(X_train, y_train)

In [ ]:
f1_score(rf.predict(X_test), y_test)